In [10]:
import os
import re
import jinja2

def get_sql_file_names():
    current_directory = os.getcwd()

    all_files = os.listdir(current_directory)

    sql_files_without_extension = []

    for file_name in all_files:
        if file_name.endswith(".sql"):
            sql_file_name = file_name[:-4]
            sql_files_without_extension.append(sql_file_name)

    return sql_files_without_extension

def get_sql_content(file_name):
    with open(f'{file_name}.sql', 'r') as sql_file:
        sql_content = sql_file.read()

    return sql_content

def get_table_columns(sql_content):
    select_statements = re.findall(r'SELECT(.*?)(FROM|$)', sql_content, re.DOTALL | re.IGNORECASE)
    if select_statements:
        last_select = select_statements[-1][0]
    else:
        last_select = ""

    pattern = r'(\w+)(?:\s+AS\s+(\w+))?(?:,|\s|$)'
    columns = re.findall(pattern, last_select)
    columns = [(column.strip(), alias.strip() if alias else column.strip()) for column, alias in columns]

    column_data = [{'name': col[1], 'description': "", 'tests': []} for col in columns]

    add_column_tests(column_data)

    return column_data

def add_column_tests(column_data):
    unique_test_added = False

    for column in column_data:
        if column['name'].endswith(('sk', 'fk', 'id')):
            if not unique_test_added:
                column['tests'].extend(['not_null', 'unique'])
                unique_test_added = True
            else:
                column['tests'].append('not_null')

def write_to_yaml_file(file_name, column_data):
    template = jinja2.Template('''\
version: 2
models:
  - name: {{ file_name }}
    description: "Information on {{ file_name }}"
    columns:{% for col in column_data %}
      - name: {{ col['name'] }}
        description: "{{ col['description'] }}"{% if col['tests'] %}
        tests:{% for test in col['tests'] %}
          - {{ test }}{% endfor %}{% endif %}{% endfor %}
    ''')

    with open(f'{file_name}.yml', 'w') as yml_file:
        yml_file.write(template.render(file_name=file_name, column_data=column_data))

    print(f"YAML file named {file_name}.yml generated successfully.")

def main():
    sql_files = get_sql_file_names()
    for file_name in sql_files:
        sql_content = get_sql_content(file_name)
        column_data = get_table_columns(sql_content)
        write_to_yaml_file(file_name, column_data)

if __name__ == "__main__":
    main()


YAML file named int_sales__order_itens.yml generated successfully.
